In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.set_printoptions(precision=4, suppress=True)

from collections import Counter
from Levenshtein import distance as levenshtein_distance

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('axes', **{'titlesize': '16', 'labelsize': '16'})
rc('legend', **{'fontsize': '16'})
rc('figure', **{'dpi' : 200})

## Наивный байесовский классификатор

In [ ]:
import codecs
with codecs.open('/home/snikolenko/soft/russian_news_corpus/FullWikiNews.csv', 'r', 'cp1251') as f:
    all_lines = f.readlines()

In [ ]:
all_data = pd.read_csv('/home/snikolenko/soft/russian_news_corpus/FullWikiNews.csv', encoding='cp1251')

In [ ]:
all_data.head()

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

In [ ]:
lemmatize(all_data["text"][0])

In [ ]:
import re
def preprocess_text(s):
    res, labels = '', []
    m, n = re.split(r'\[\[[^\]]*\]\]', s), re.findall(r'\[\[[^\]]*\]\]', s)
    for i,x in enumerate(n):
        res += m[i]
        if x[2:11] != 'категория':
            res += x.split('|')[-1][:-2]
        else:
            labels.append(x.split(':')[-1][:-2])
    res = re.sub(r'\{\{[^\}]*\}\}', '', res).strip()
    res = re.sub(r'\|[^|]*\|', '', res).strip()
    res = re.sub(r'\.', ' ', res).strip()
    res = re.sub(r'[*—»«]', '', res).strip()
    return res, labels

In [ ]:
texts, labels = [], []
for s in all_data["text"]:
    t, ls = preprocess_text(s)
    texts.append(t)
    labels.append(ls)

In [ ]:
clabels = Counter([l for ls in labels for l in ls])
print(clabels.most_common(20))

classes = ['политика', 'экономика', 'происшествия', 'культура', 'наука и технологии']
classes_in_labels = [ np.where([x in ls for x in classes])[0] for ls in labels ]
Xtext_full = [x for i,x in enumerate(texts) if len(classes_in_labels[i])>0]
y = np.array([np.max(s) for s in classes_in_labels if len(s) > 0])

In [ ]:
import pickle

# Xtext = []
# for i,t in enumerate(Xtext_full):
#     if i % 100 == 0:
#         print('%d...' % i)
#     Xtext.append( lemmatize(t) )

# with open('lemtexts.pkl', 'wb') as f:
#     pickle.dump(Xtext, f)

with open('lemtexts.pkl', 'rb') as f:
    Xtext = pickle.load(f)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
test_set_size = 100
test_set = np.sort(np.random.choice(len(y), size=test_set_size))
train_set = np.array([ i for i in range(len(y)) if not y in test_set ])

def accuracy(y_pred, y_true):
    return sum(y_pred == y_true) / len(y_true)

In [ ]:
range_maxdf = np.arange(0.05, 0.5, 0.05)
range_mindf = np.arange(0., 0.02, 0.001)

results = {}
print("min_df\tmax_df\tFeatures\tBernoulliNB\tMultinomialNB")
for max_df in range_maxdf:
    for min_df in range_mindf:
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df)
        X_train = vectorizer.fit_transform([ " ".join(s) for s in np.array(Xtext)[train_set]])
        X_test = vectorizer.transform([ " ".join(s) for s in np.array(Xtext)[test_set]])
        y_train, y_test = y[train_set], y[test_set]

        model_nbm, model_nbb = MultinomialNB(fit_prior=True), BernoulliNB(fit_prior=True)
        model_nbm.fit(X_train, y_train)
        model_nbb.fit(X_train, y_train)
        acc_nbb, acc_nbm = accuracy(model_nbb.predict(X_test), y_test), accuracy(model_nbm.predict(X_test), y_test)
        print("%.4f\t%.4f\t%d\t%.4f\t%.4f" % (min_df, max_df, len(vectorizer.vocabulary_), acc_nbb, acc_nbm))
        results[(min_df, max_df)] = (acc_nbb, acc_nbm, len(vectorizer.vocabulary_))

In [ ]:
res_nbb_toplot = np.array([[ results[(min_df, max_df)][0] for min_df in range_mindf ] for max_df in range_maxdf ] )

fig = plt.figure(figsize=(15,12))
ax = sns.heatmap(res_nbb_toplot, annot=True)
ax.set_xticklabels(["%.3f" % min_df for min_df in range_mindf ])
ax.set_yticklabels(["%.2f" % max_df for max_df in range_maxdf ])
plt.show()

In [ ]:
min_df, max_df = 0.0050, 0.0500
vectorizer = CountVectorizer(min_df=min_df, max_df=max_df)
X_train_fulltext = np.array(Xtext_full)[train_set]
X_train = vectorizer.fit_transform([ " ".join(s) for s in np.array(Xtext)[train_set]])

N, M, K = X_train.shape[0], X_train.shape[1], len(classes)

## EM-алгоритм для кластеризации

In [ ]:
print(N, M, K)
X = X_train.toarray()

In [ ]:
def e_step(X, probs):
    z = np.matmul( X, np.log(probs).T )
    z = z - np.logaddexp.reduce(z, axis=1).reshape(-1, 1)
    return np.exp(z)

def m_step(X, z):
    probs = ( np.matmul(X.T, z) + np.ones((M, K)) ).T
    probs = np.divide( probs, np.sum(probs, axis=1).reshape(-1, 1) )
    return probs

def log_likelihood(X, z, probs):
    return np.sum(np.multiply( z, np.log(np.matmul( X, probs.T )) ))

In [ ]:
probs = np.ones((K, M))
random_sample = np.random.choice(N, replace=False, size=5*K)
for k in range(K):
    probs[k] = probs[k] + np.sum(X[random_sample[5*k:5*k+5]], axis=0)
probs = np.divide( probs, np.sum(probs, axis=1).reshape(-1,1) )

In [ ]:
old_l = 0
for iIter in range(100):
    new_z = e_step(X, probs)
    new_probs = m_step(X, new_z)
    l = log_likelihood(X, new_z, new_probs)
    print("Итерация %d:\t\t%.4f" % (iIter, l))
    if np.abs(l - old_l) < 1e-4:
        break
    z, probs, old_l = new_z, new_probs, l

In [ ]:
for iCluster in range(5):
    print("=== Кластер %d ===\n" % iCluster)
    for iDoc in np.where(z[:, iCluster] > 0.99)[0][:10]:
        print(X_train_fulltext[iDoc][:300] + "\n\n")

## pLSI

In [ ]:
T = 10
Phi, Theta = np.random.random(size=(T, M)), np.random.random(size=(N, T))
Phi = np.divide( Phi, np.sum(Phi, axis=1).reshape(-1, 1) )
Theta = np.divide( Theta, np.sum(Theta, axis=1).reshape(-1, 1) )

In [ ]:
def e_step(X, Phi, Theta):
    n_t, n_dt, n_wt = np.zeros(T), np.zeros((N, T)), np.zeros((M, T))
    for i in range(X.shape[0]):
        n_tdw = np.divide( np.multiply( np.multiply( Phi, Theta[i].reshape(-1, 1) ), X[i].reshape(1, -1)), np.matmul( Theta[i], Phi ).reshape(1, -1) )
        n_dt[i] = np.sum( n_tdw, axis=1 )
        n_t += np.sum( n_tdw, axis=1 )
        n_wt += n_tdw.T
    return n_wt, n_dt, n_t

def m_step(X, n_wt, n_dt, n_t):
    new_Phi = np.divide( n_wt, np.sum(n_wt, axis=0) ).T
    new_Theta = np.divide( n_dt, np.sum(X, axis=1).reshape(-1, 1) )
    return new_Phi, new_Theta

In [ ]:
for iIter in range(100):
    n_wt, n_dt, n_t = e_step(X, Phi, Theta)
    new_Phi, new_Theta = m_step(X, n_wt, n_dt, n_t)
    
    residue = np.sum( (Phi - new_Phi) ** 2 ) + np.sum( (Theta - new_Theta) ** 2 )
    print("Итерация %d:\t\t%.4f" % (iIter, residue))
    if residue < 1e-3:
        break
    Phi, Theta = new_Phi, new_Theta

In [ ]:
voc = [x[0] for x in sorted([(w,i) for w,i in vectorizer.vocabulary_.items()], key=lambda x: x[1]) ]

In [ ]:
for t in range(T):
    word_probs = sorted([ (x, Phi[t, i]) for i,x in enumerate(voc) ], key=lambda x: x[1], reverse=True)
    print("Тема %d\n%s\n\n" % (t, "\n".join([ "%20s\t%.5f" % (x[0], x[1]) for x in word_probs[:10]])))

## LDA

In [ ]:
T = 10
Phi = np.random.random(size=(M, T))
Phi = np.divide( Phi, np.sum(Phi, axis=0).reshape(1, -1) )
Theta = np.random.random(size=(N, T))
Theta = np.divide( Theta, np.sum(Theta, axis=1).reshape(-1, 1) )

In [ ]:
alpha = (2./T) * np.ones(T)

In [ ]:
doc_lengths = np.sum(X, axis=1)
doc_words = []
for x in X:
    dw = []
    for i in np.where(x > 0)[0]:
        for _ in range(x[i]):
            dw.append(i)
    doc_words.append(np.array(dw))

In [ ]:
def internal_em(doc_words, lPhi):
    Gamma = np.random.random(size=(N, T))
    Pi = [ np.zeros((n, T)) for n in doc_lengths ]
    for iIter in range(100):
        all_digammas = sp.special.digamma(Gamma) - sp.special.digamma(np.sum(Gamma, axis=1)).reshape(-1, 1)
        for i,dw in enumerate(doc_words):
            Pi[i] = lPhi[doc_words[i]] + all_digammas[i]
            Pi[i] = np.exp(Pi[i] - np.logaddexp.reduce(Pi[i], axis=1).reshape(-1, 1))
        new_Gamma = np.vstack( [alpha + np.sum(dpi, axis=0) for dpi in Pi] )
        residue = np.sum((new_Gamma-Gamma) ** 2)
        Gamma = new_Gamma
        if residue < 0.01:
            break
    return Gamma, Pi

In [ ]:
def phi_m_step(doc_words, Pi):
    Phi = np.zeros((M, T))
    for i,dw in enumerate(doc_words):
        Phi[ dw ] += Pi[i]
    Phi = np.divide( Phi, np.sum(Phi, axis=0).reshape(1, -1) )
    return Phi

In [ ]:
for iIter in range(20):
    lPhi = np.log(Phi)
    Gamma, Pi = internal_em(doc_words, lPhi)
    new_Phi = phi_m_step(doc_words, Pi)
    residue = np.sum( (new_Phi - Phi) ** 2)
    Phi = new_Phi
    print("Итерация %d\t\t%.7f" % (iIter, residue))
    if residue < 1e-5:
        break

In [ ]:
for t in range(T):
    word_probs = sorted([ (x, Phi[i, t]) for i,x in enumerate(voc) ], key=lambda x: x[1], reverse=True)
    print("Тема %d\n%s\n\n" % (t, "\n".join([ "%20s\t%.5f" % (x[0], x[1]) for x in word_probs[:10]])))

In [ ]:
beta = (50./M) * np.ones(M)

In [ ]:
Gamma = np.random.random(size=(N, T))
Pi = [ np.zeros((n, T)) for n in doc_lengths ]
Lambda = np.random.random(size=(M, T))
for iIter in range(200):
    Phi = np.divide( Lambda, np.sum(Lambda, axis=0).reshape(1, -1) )
    lPhi = np.log(Phi)
    all_digammas = sp.special.digamma(Gamma) - sp.special.digamma(np.sum(Gamma, axis=1)).reshape(-1, 1)
    for i,dw in enumerate(doc_words):
        Pi[i] = lPhi[doc_words[i]] + all_digammas[i]
        Pi[i] = np.exp(Pi[i] - np.logaddexp.reduce(Pi[i], axis=1).reshape(-1, 1))
    new_Gamma = np.vstack( [alpha + np.sum(dpi, axis=0) for dpi in Pi] )
    new_Lambda = np.zeros((M, T)) + beta.reshape(-1, 1)
    for i,dw in enumerate(doc_words):
        new_Lambda[ dw, : ] += Pi[i]
    residue = ( np.sum((new_Gamma-Gamma) ** 2), np.sum((new_Lambda-Lambda) ** 2) )
    print("Итерация %d\t\t%.5f\t%.5f" % (iIter, residue[0], residue[1]))
    Gamma, Lambda = new_Gamma, new_Lambda
    if residue[0] < 1e-2 and residue[1] < 1e-2:
        break

In [ ]:
Phi = np.divide( Lambda, np.sum(Lambda, axis=0).reshape(1, -1) )
for t in range(T):
    word_probs = sorted([ (x, Phi[i, t]) for i,x in enumerate(voc) ], key=lambda x: x[1], reverse=True)
    print("Тема %d\n%s\n\n" % (t, "\n".join([ "%20s\t%.5f" % (x[0], x[1]) for x in word_probs[:10]])))